Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайта superjob.ru и hh.ru. Приложение должно анализировать несколько страниц сайта(также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:
*Наименование вакансии
*Предлагаемую зарплату (отдельно мин. и отдельно макс. и отдельно валюта)
*Ссылку на саму вакансию        
*Сайт откуда собрана вакансия

По своему желанию можно добавить еще работодателя и расположение. Данная структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas.

1) Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, записывающую собранные вакансии в созданную БД
2) Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введенной суммы. Поиск по двум полям (мин и макс зарплату)
3) Написать функцию, которая будет добавлять в вашу базу данных только новые вакансии с сайта

In [38]:
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd
import json
from pymongo import MongoClient

In [39]:
from pprint import  pprint

class insert_to_mongodb():
    def __init__(self, my_db,my_collecton, connect_str='localhost:27017'):
        self.connect_str=connect_str
        self.my_db=my_db
        self.my_collection=my_collecton
    
    def __connect(self):
        try:
            client = MongoClient(self.connect_str)
            db = client[self.my_db]
            collection = db[self.my_collection]
            return  collection
        except Exception as e:
            print("нет подключения" + str(e))

    def only_insert_all_data(self,my_data):
        cursor=self.__connect()
        cursor.insert_many(my_data)

    def search_vacancies_by_amount(self,amount):
        cursor = self.__connect()
        result = cursor.find({'$or': [ { 'Зарплата мин': { '$gte': amount } }, {'Зарплата макс': { '$gte': amount } } ] },{'_id':0})
        for i in result:
            pprint(i)

    def add_new(self,my_data):
        cursor = self.__connect()
        for i in my_data:
            cursor.update_one({'link':i['link']},{'$set':i},upsert=True)


In [40]:
class hh():
    def __init__(self,s_key):
        self.mian_link ='https://lipetsk.hh.ru'
        self.s_key=s_key

    def get_vacancies(self):
        url = '/search/vacancy'
        params = {'clusters=true': 'true', 'enable_snippets': 'true', 'text': self.s_key,
                  'only_with_salary': 'true', 'from': 'cluster_compensation',
                  'showClusters': 'true'}
        header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.101 YaBrowser/20.7.0.1230 Yowser/2.5 Safari/537.36',
            'Accept': '*/*'}
        response = requests.get(self.mian_link + url, headers=header, params=params)
        soup = bs(response.text, 'lxml')
        my_data = soup.find_all('div', {'class': "vacancy-serp-item"})
        my_list = []
        self.__get_info_from_page(my_data,my_list)
        page = soup.find('script', {'data-name': 'HH/Pager'})
        num_pages=json.loads(page['data-params'])['pagesCount']
        while num_pages > 1:
            params = {'clusters=true': 'true', 'enable_snippets': 'true', 'text': self.s_key,
                  'only_with_salary': 'true', 'from': 'cluster_compensation',
                  'showClusters': 'true','page':str((num_pages-1))}
            response = requests.get(self.mian_link + url, headers=header, params=params)
            soup = bs(response.text, 'lxml')
            my_data = soup.find_all('div', {'class': "vacancy-serp-item"})
            self.__get_info_from_page(my_data, my_list)
            num_pages-=1
        return my_list

    def __get_info_from_page(self,my_data,my_list):
        for i in my_data:
            my_dict = {}
            my_dict['Позиция'] = i.find('a', {'class': 'HH-LinkModifier'}).getText()
            my_dict['link'] = i.find('a', {'class': 'HH-LinkModifier'})['href']
            if i.find('a', {'data-qa': 'vacancy-serp__vacancy-employer'}):
                my_dict['Организация'] = i.find('a', {'data-qa': 'vacancy-serp__vacancy-employer'}).text
            else:
                my_dict['Организация'] = "No Data"
            if i.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'}):
                salary = self.__get_salary(i.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'}).text)
            my_dict['Зарплата мин'] = salary['min']
            my_dict['Зарплата макс'] = salary['max']
            my_dict['Валюта'] = salary['currency']
            my_dict['Ресурс'] = self.mian_link
            my_list.append(my_dict)

    def __get_salary(self,str):
          salary = {}
          if str[:2] == 'от':
              min = re.search(r'[\d\s]+', str)[0]
              min = int(re.sub(r"\s", '', min))
              max = None
          elif str[:2] == 'до':
              max = re.search(r'[\d\s]+', str)[0]
              max = int(re.sub(r"\s", '', max))
              min = None
          else:
              min = re.search(r'[\d\s]+', str)[0]
              min = int(re.sub(r"\s", '', min))
              max = re.search(r'-([\d\s]+)', str)[1]
              max = int(re.sub(r"\s", '', max))
          currency = re.search(r'[A-Zа-я]{3}', str)[0]
          salary['min'] = min
          salary['max'] = max
          salary['currency'] = currency
          return salary

In [41]:
class sj():
    def __init__(self,s_key):
        self.mian_link ='https://superjob.ru'
        self.s_key=s_key

    def get_vacancies_from_with_salary(self):
        url = '/vacancy/search'
        params = { 'keywords': self.s_key, 'payment_defined':'1'}
        header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.101 YaBrowser/20.7.0.1230 Yowser/2.5 Safari/537.36',
            'Accept': '*/*'}
        response = requests.get(self.mian_link + url, headers=header, params=params)
        soup = bs(response.text, 'lxml')
        my_data = soup.find_all('div', {'class': "iJCa5 f-test-vacancy-item _1fma_ _1JhPh _2gFpt _1znz6 _2nteL"})
        my_list = []
        self.__get_info_from_page(my_data, my_list)
        while soup.find('a', {'rel': 'next'}):
            page=soup.find('a', {'rel': 'next'})['href']
            params = { 'keywords': self.s_key, 'payment_defined':'1'}
            response = requests.get(self.mian_link +page, headers=header, params=params)
            soup = bs(response.text, 'lxml')
            my_data = soup.find_all('div', {'class': "iJCa5 f-test-vacancy-item _1fma_ _1JhPh _2gFpt _1znz6 _2nteL"})
            self.__get_info_from_page(my_data, my_list)
        return my_list

    def __get_info_from_page(self, my_data, my_list):
        for i in my_data:
            my_dict = {}
            my_dict['Позиция'] = i.find('a').getText()
            my_dict['link'] = self.mian_link+i.find('a')['href']
            my_dict['Организация'] = i.find('a', {'class':'_25-u7'}).text
            sal=i.find('span', {'class': 'PlM3e'}).text
            salary = self.__get_salary(sal)
            my_dict['Зарплата мин'] = salary['min']
            my_dict['Зарплата макс'] = salary['max']
            my_dict['Валюта'] = salary['currency']
            my_dict['Ресурс'] = self.mian_link
            my_list.append(my_dict)
        # print(my_dict)

    def __get_salary(self, str):
        salary = {}
        if str[:2] == 'от':
            min = re.search(r'[\d\s]+', str)[0]
            min = int(re.sub(r"\s", '', min))
            max = None
            currency = re.search(r'[A-Zа-я]{3}', str)[0]
        elif str[:2] == 'до':
            max = re.search(r'[\d\s]+', str)[0]
            max = int(re.sub(r"\s", '', max))
            min = None
            currency = re.search(r'[A-Zа-я]{3}', str)[0]
        elif '-' in str:
            min = re.search(r'[\d\s]+', str)[0]
            min = int(re.sub(r"\s", '', min))
            max = re.search(r'-([\d\s]+)', str)[1]
            max = int(re.sub(r"\s", '', max))
            currency = re.search(r'[A-Zа-я]{3}', str)[0]
        elif str=="По договорённости":
            min=None
            max=None
            currency = None
        else:
            min = re.search(r'[\d\s]+', str)[0]
            min = int(re.sub(r"\s", '', min))
            max = re.search(r'[\d\s]+', str)[0]
            max = int(re.sub(r"\s", '', max))
            currency = re.search(r'[A-Zа-я]{3}', str)[0]
        salary['min'] = min
        salary['max'] = max
        salary['currency'] = currency
        return salary

In [42]:
vac_hh=hh('Data')
vac_sj = sj('Data')

In [43]:
my_data = vac_hh.get_vacancies()+vac_sj.get_vacancies_from_with_salary()
df = pd.DataFrame(my_data)

In [44]:
df

,Позиция,link,Организация,Зарплата мин,Зарплата макс,Валюта,Ресурс
0,Data Engineer (Product Analytics Platform),https://lipetsk.hh.ru/vacancy/37856356?query=Data,JetBrains,180000.0,NaN,руб,https://lipetsk.hh.ru
1,Head of Data,https://lipetsk.hh.ru/vacancy/38193065?query=Data,Наталья Мирская,600000.0,800000.0,руб,https://lipetsk.hh.ru
2,Head of Data,https://lipetsk.hh.ru/vacancy/38075193?query=Data,ООО Неолаб-Нск,200000.0,NaN,руб,https://lipetsk.hh.ru
3,Data Scientist (Computer Vision),https://lipetsk.hh.ru/vacancy/38090271?query=Data,Tevian (ООО Технологии видеоанализа),90000.0,200000.0,руб,https://lipetsk.hh.ru
4,Data Scientist,https://lipetsk.hh.ru/vacancy/37782747?query=Data,ООО СКАЙТРЭК,120000.0,180000.0,руб,https://lipetsk.hh.ru
...,...,...,...,...,...,...,...
300,Старший системный администратор (DevOps),https://superjob.ru/vakansii/starshij-sistemny...,Data master,120000.0,120000.0,руб,https://superjob.ru
301,Python Developer,https://superjob.ru/vakansii/python-developer-...,Грандстафф,160000.0,160000.0,руб,https://superjob.ru
302,Ведущий программист 1С-Битрикс,https://superjob.ru/vakansii/veduschij-program...,Студия Клондайк,NaN,150000.0,руб,https://superjob.ru
303,"Главный специалист (пиар, контент)",https://superjob.ru/vakansii/glavnyj-specialis...,Территориальный центр социального обслуживания...,50000.0,50000.0,руб,https://superjob.ru


In [45]:
if __name__=='__main__': 
    load  = insert_to_mongodb('my_db', 'test')
    load.only_insert_all_data(my_data)
    load.search_vacancies_by_amount(800000)
    try:
        load.add_new(my_data) 
    except:
        print('End')

{'Link': 'https://lipetsk.hh.ru/vacancy/38193065?query=Data',
 'Валюта': 'руб',
 'Зарплата макс': 800000,
 'Зарплата мин': 600000,
 'Организация': ' Наталья Мирская',
 'Позиция': 'Head of Data',
 'Ресурс': 'https://lipetsk.hh.ru'}
{'Link': 'https://lipetsk.hh.ru/vacancy/38159690?query=Data',
 'Валюта': 'KZT',
 'Зарплата макс': 900000,
 'Зарплата мин': 650000,
 'Организация': 'ТОО One technologies',
 'Позиция': 'ML Developer/Data scientist',
 'Ресурс': 'https://lipetsk.hh.ru'}
{'Link': 'https://lipetsk.hh.ru/vacancy/38193065?query=Data',
 'Валюта': 'руб',
 'Зарплата макс': 800000,
 'Зарплата мин': 600000,
 'Организация': ' Наталья Мирская',
 'Позиция': 'Head of Data',
 'Ресурс': 'https://lipetsk.hh.ru'}
{'Link': 'https://lipetsk.hh.ru/vacancy/38159690?query=Data',
 'Валюта': 'KZT',
 'Зарплата макс': 900000,
 'Зарплата мин': 650000,
 'Организация': 'ТОО One technologies',
 'Позиция': 'ML Developer/Data scientist',
 'Ресурс': 'https://lipetsk.hh.ru'}
{'Link': 'https://lipetsk.hh.ru/vacanc

 'Валюта': 'руб',
 'Зарплата макс': 800000,
 'Зарплата мин': 600000,
 'Организация': ' Наталья Мирская',
 'Позиция': 'Head of Data',
 'Ресурс': 'https://lipetsk.hh.ru'}
{'link': 'https://lipetsk.hh.ru/vacancy/38159690?query=Data',
 'Валюта': 'KZT',
 'Зарплата макс': 900000,
 'Зарплата мин': 650000,
 'Организация': 'ТОО One technologies',
 'Позиция': 'ML Developer/Data scientist',
 'Ресурс': 'https://lipetsk.hh.ru'}
{'link': 'https://lipetsk.hh.ru/vacancy/38193065?query=Data',
 'Валюта': 'руб',
 'Зарплата макс': 800000,
 'Зарплата мин': 600000,
 'Организация': ' Наталья Мирская',
 'Позиция': 'Head of Data',
 'Ресурс': 'https://lipetsk.hh.ru'}
{'link': 'https://lipetsk.hh.ru/vacancy/38159690?query=Data',
 'Валюта': 'KZT',
 'Зарплата макс': 900000,
 'Зарплата мин': 650000,
 'Организация': 'ТОО One technologies',
 'Позиция': 'ML Developer/Data scientist',
 'Ресурс': 'https://lipetsk.hh.ru'}
{'link': 'https://lipetsk.hh.ru/vacancy/38193065?query=Data',
 'Валюта': 'руб',
 'Зарплата макс': 80

Делалась база на три профессии ( 'Data', 'хирург', 'рабочий')
Новые данные дописываются в конец, поиск ведётся корректно для всех профессий